In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn-whitegrid") # seabornの背景色を白に変更

### データ生成
3つの都市におけるアパートの家賃のデータセット
- Rent : 家賃
- City : 都市の名前

In [2]:
df = pd.DataFrame({
    'City': ['SF', 'SF', 'SF', 'NYC', 'NYC', 'NYC', 'Seattle', 'Seattle', 'Seattle'],
    'Rent': [3999, 4000, 4001, 3499, 3500, 3501, 2499, 2500, 2501]
})

## エンコーディング
質的データは通常数値で表されていないため, 数値データに変換する必要がある. この変換をエンコーディングという. ここではOne-Hotエンコーディング, ダミーエンコーディング, Effectエンコーディングの3つを行う.

### One-Hotエンコーディング
One-Hotエンコーディングはビットのグループを利用してエンコーディングを行う方法である. 質的変数が複数のカテゴリに属さない場合, グループ内の1つのビットのみを1, それ以外を0というふうに変換を行う.

In [3]:
# prefix : エンコーディングするカラムを指定
one_hot_df = pd.get_dummies(df,prefix=["city"])
one_hot_df

,Rent,city_NYC,city_SF,city_Seattle
0,3999,0,1,0
1,4000,0,1,0
2,4001,0,1,0
3,3499,1,0,0
4,3500,1,0,0
5,3501,1,0,0
6,2499,0,0,1
7,2500,0,0,1
8,2501,0,0,1


### ダミーエンコーディング
One-Hotエンコーディングはk個のカテゴリを割り当てているが,実際には最後の1カテゴリは「他全てのビットが0」という風に割り当てることができる. またOne-Hotエンコーディングで生成されたビット間には線形依存の関係があるため,モデルの係数が一意に定まらない(多重共線性)という問題がある. ダミーエンコーディングは最後の1カテゴリを「他全てのビットが0」という風に割り当てることで自由度をk-1にして,余分なビットを取り除いている.

In [4]:
# drop_first=Trueにすることでダミーエンコーディングが行われる
dummy_df = pd.get_dummies(df,prefix=["city"],drop_first=True)
dummy_df

,Rent,city_SF,city_Seattle
0,3999,1,0
1,4000,1,0
2,4001,1,0
3,3499,0,0
4,3500,0,0
5,3501,0,0
6,2499,0,1
7,2500,0,1
8,2501,0,1


### Effectエンコーディング
Effectエンコーディングは参照カテゴリに-1を割り当てるエンコーディング方法である. 参照カテゴリの説明はhttps://uribo.github.io/practical-ds/02/categorical.html がわかりやすい. Effectエンコーディングでは切片はターゲット全体の平均を表し, 各特徴量の回帰係数は各カテゴリの平均値と全体の差分を表す. 個の差分は各カテゴリの主成分(main effect)と呼ばれる.

In [9]:
effect_df = dummy_df.copy()
# ダミーエンコーディングで型がunsigned intになるためint64に変換
effect_df = effect_df[["city_SF","city_Seattle"]].astype("int64")
# 3,4,5レコードに-1を代入
effect_df.loc[3:5,["city_SF","city_Seattle"]] = -1
effect_df

,city_SF,city_Seattle
0,1,0
1,1,0
2,1,0
3,-1,-1
4,-1,-1
5,-1,-1
6,0,1
7,0,1
8,0,1


ダミーエンコーディングからEffectエンコーディングへの変換は,参照カテゴリのレコードを全て指定する必要があり膨大なデータの場合は面倒くさい. そこでOne-Hotエンコーディングから参照カテゴリのビットが立っている他のダミー変数の値を-1で上書きすることでレコードの指定を回避する.

In [20]:
drop_column = "city_NYC" # 参照カテゴリを選択

# One-Hotエンコーディング
one_hot_df = pd.get_dummies(df,prefix=["city"])
# 型変換
one_hot_df[["city_SF","city_Seattle"]] = one_hot_df[["city_SF","city_Seattle"]].astype("int64")
# 参照カテゴリのビットが立っている他のダミー変数の値を-1で上書き
one_hot_df.loc[one_hot_df[drop_column]==1,["city_SF","city_Seattle"]] = -1
# 参照カテゴリを除去
effect_df = one_hot_df.drop(columns=[drop_column])
effect_df

,Rent,city_SF,city_Seattle
0,3999,1,0
1,4000,1,0
2,4001,1,0
3,3499,-1,-1
4,3500,-1,-1
5,3501,-1,-1
6,2499,0,1
7,2500,0,1
8,2501,0,1


## 膨大なカテゴリ数を持つカテゴリ変数の扱い
膨大なカテゴリ数を持つ特徴量をエンコーディングする手法として特徴量ハッシングとビンカウンティングを行う．

### 特徴量ハッシング
特徴量ハッシングは膨大なカテゴリ数を扱う手法の一つである. 潜在的に無限に広がる特徴量IDをハッシュ関数を通して有限のm次元ベクトルに割り当てることで特徴量を圧縮する. mは特徴量ハッシング後の内積の誤差が許容可能になるように調整される. この手法は容量が小さくなる反面, 直観的な解釈が難しくなるためｎ, 解釈を目的としたデータ探索や可視化で適用することは好ましくない. ここではレストランのレビューデータを用いて特徴量ハッシングを行う.

In [24]:
# load data
import json
# 最初の10,000件のレビューを読み込み
with open('./Data/yelp_academic_dataset_review.json',encoding="utf-8_sig") as f:
    js = []
    for i in range(10000):
        js.append(json.loads(f.readline()))

review_df = pd.DataFrame(js)
review_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,lWC-xP3rd6obsecCYsGZRg,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4.0,3,1,1,Apparently Prides Osteria had a rough summer a...,2014-10-11 03:34:02
1,8bFej1QE5LXp4O05qjGqXA,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,4.0,1,0,0,This store is pretty good. Not as great as Wal...,2015-07-03 20:38:25
2,NDhkzczKjLshODbqDoNLSg,eC5evKn1TWDyHCyQAwguUw,_sS2LBIGNT5NQb6PD1Vtjw,5.0,0,0,0,I called WVM on the recommendation of a couple...,2013-05-28 20:38:06
3,T5fAqjjFooT4V0OeZyuk1w,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,2.0,1,1,1,I've stayed at many Marriott and Renaissance M...,2010-01-08 02:29:15
4,sjm_uUcQVxab_EeLCqsYLg,0kA0PAJ8QFMeveQWHFqz2A,8zehGz9jnxPqXtOc7KaJxA,4.0,0,0,0,The food is always great here. The service fro...,2011-07-28 18:05:01


In [27]:
# mにbusiness_idのユニーク数を代入
m = len(review_df['business_id'].unique())
print(m)

# 特徴量ハッシング
from sklearn.feature_extraction import FeatureHasher
h = FeatureHasher(n_features=m,input_type="string")
f = h.transform(review_df["business_id"])

# business_id
print(review_df["business_id"].unique().tolist()[0:5])
# business_id after hashing
print(f.toarray())

4299
['buF9druCkbuXLX526sGELQ', 'RA4V8pr014UyUbDvI-LW2A', '_sS2LBIGNT5NQb6PD1Vtjw', '0AzLzHfOJgL7ROwhdww2ew', '8zehGz9jnxPqXtOc7KaJxA']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### ビンカウンティング
ビンカウンティングはカテゴリごとになんらかの統計量を計算して,これを特徴量として利用するという手法である. ここでは広告の情報からクリック率を予測するタスクを想定してビンカウンティングを行う.

In [2]:
# 6GBあるからメモリに注意
df = pd.read_csv("Data/avazu_train.csv")
df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157


In [3]:
# device_idが何種類あるか計算
print(len(df["device_id"].unique()))

2686408


In [4]:
def click_counting(x, bin_column):
    """
    各カテゴリに対して次を計算する関数
    Theta = [counts,p(click),p(no click),p(click)/p(no click)]
    counts : クリック回数
    p(click) : クリック率
    p(no click) : 1-クリック率
    p(click)/p(no click) : クリック率と非クリック率の非
    """
    
    # clicks    
    clicks = pd.Series(x[x['click'] > 0][bin_column].value_counts(), name='clicks')
    # no clicks
    no_clicks = pd.Series(x[x['click'] < 1][bin_column].value_counts(), name='no_clicks')
    # counts
    counts = pd.DataFrame([clicks,no_clicks]).T.fillna('0')
    counts['total_clicks'] = counts['clicks'].astype('int64') + counts['no_clicks'].astype('int64')
    return counts

def bin_counting(counts):
    # ビンカウンティングを計算する関数
    counts['N+'] = counts['clicks'].astype('int64').divide(counts['total_clicks'].astype('int64'))
    counts['N-'] = counts['no_clicks'].astype('int64').divide(counts['total_clicks'].astype('int64'))
    counts['log_N+'] = counts['N+'].divide(counts['N-'])
    # Bin Countingのプロパティを返すだけの場合、ここでフィルタリングを実行
    bin_counts = counts.filter(items= ['N+', 'N-', 'log_N+'])
    return counts, bin_counts

# device_idを対象としたビンカウンティング
bin_column = 'device_id'
device_clicks = click_counting(df.filter(items=[bin_column, 'click']), bin_column)
device_all, device_bin_counts = bin_counting(device_clicks)

device_all

,clicks,no_clicks,total_clicks,N+,N-,log_N+
a99f214a,5.80942e+06,2.75489e+07,33358308,0.174152,0.825848,0.210877
0f7c61dc,16190,5166,21356,0.758101,0.241899,3.133953
c357dbff,12469,7198,19667,0.634006,0.365994,1.732287
3cdb4052,2421,648,3069,0.788856,0.211144,3.736111
afeffc18,2211,7443,9654,0.229024,0.770976,0.297058
...,...,...,...,...,...,...
79aad06a,0,1,1,0.000000,1.000000,0.000000
4bcc2786,0,1,1,0.000000,1.000000,0.000000
5994c002,0,1,1,0.000000,1.000000,0.000000
7ba50562,0,1,1,0.000000,1.000000,0.000000


## featuretools

In [2]:
import featuretools as ft

### aggregation

In [3]:
data = {'item_id': [1, 2, 3, 4, 5],
        'name': ['apple', 'broccoli', 'cabbage', 'dorian', 'eggplant'],
        'category': ['fruit', 'vegetable', 'vegetable', 'fruit', 'vegetable'],
        'price': [100, 200, 300, 4000, 500]}
item_df = pd.DataFrame(data)
item_df

,item_id,name,category,price
0,1,apple,fruit,100
1,2,broccoli,vegetable,200
2,3,cabbage,vegetable,300
3,4,dorian,fruit,4000
4,5,eggplant,vegetable,500


In [4]:
# EntitySetの作成
es = ft.EntitySet(id='example')
es = es.entity_from_dataframe(entity_id='items',
                              dataframe=item_df,
                              index='item_id')
es

Entityset: example
  Entities:
    items [Rows: 5, Columns: 4]
  Relationships:
    No relationships

In [5]:
es = es.normalize_entity(base_entity_id='items',
                         new_entity_id='category',
                         index='category')
es

Entityset: example
  Entities:
    items [Rows: 5, Columns: 4]
    category [Rows: 2, Columns: 1]
  Relationships:
    items.category -> category.category

In [6]:
es['items'].df

,item_id,name,category,price
1,1,apple,fruit,100
2,2,broccoli,vegetable,200
3,3,cabbage,vegetable,300
4,4,dorian,fruit,4000
5,5,eggplant,vegetable,500


In [7]:
es['category'].df

,category
fruit,fruit
vegetable,vegetable


In [8]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                      target_entity='items',
                                      trans_primitives=[],
                                      agg_primitives=['count', 'sum', 'mean','max','min'],
                                      max_depth=2)
feature_matrix

,name,category,price,category.COUNT(items),category.MAX(items.price),category.MEAN(items.price),category.MIN(items.price),category.SUM(items.price)
item_id,,,,,,,,
1,apple,fruit,100,2,4000,2050.000000,100,4100
2,broccoli,vegetable,200,3,500,333.333333,200,1000
3,cabbage,vegetable,300,3,500,333.333333,200,1000
4,dorian,fruit,4000,2,4000,2050.000000,100,4100
5,eggplant,vegetable,500,3,500,333.333333,200,1000
